# plot_n10_cdnc.py

Code to plot N10 and CDNC time series (figure 2) from CAP and UM, and distributions of N10 (figure S1).

In [ ]:
import xarray as xr
import pandas as pn
import numpy as np
import numpy.ma as ma
import datetime as dt
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import capricorn_figure_props as fprops
from glob import glob
import capricorn_functions as cfn
import capricorn_constants as const
from importlib import reload
reload(cfn)

### Load CAP measurements from file

In [ ]:
# N10 from CPC
# DOI: https://doi.org/10.25919/2h1c-t753
filename = 'data/CAPRICORN/capricorn2_cpc3772_L2_1H.csv'
cpc_df = pn.read_csv(filename, index_col=0, parse_dates=True)
# cap_n10 = xr.DataArray(
#     data=np.ma.masked_invalid(cpc_df["cn_avg"].values),
#     coords={'time':cpc_df.index},
#     name='n10',
#     attrs={'units':'cm-3'}
# )
cap_n10 = xr.Dataset(
    data_vars={
        'n10': ('time', np.ma.masked_invalid(cpc_df["cn_avg"].values)),
        'n10_sigma': ('time', np.ma.masked_invalid(cpc_df["cn_sigma_avg"].values))
    },
    coords={'time':cpc_df.index},
    attrs={'units':'cm-3'}
)

# CDNC from radar and lidar retrievals
# loop over the files to open because xr.open_mfdataset doesn't like them
retr_files = glob('../polarres-obs/data/capricorn2/retrievals/radar_25m*')
retr_time = []
retr_cdnc = []
retr_cdnc_sigma = []
for file in retr_files:
    print('Loading', file)
    fn_fmt = '%Y%m%d'
    file_date = dt.datetime.strptime(file.split('_')[-2], fn_fmt)
    ds_file = xr.open_dataset(file, decode_times=False)
    time_file = ds_file['radar_time']
    time_points = [file_date + dt.timedelta(seconds=T*3600)
                   for T in time_file.values]
    cdnc_file = ds_file['nd_retrieved']
    cdnc_sigma_file = ds_file['nd_uncertainty']
    retr_time.extend(time_points)
    retr_cdnc.extend(cdnc_file.values)
    retr_cdnc_sigma.extend(cdnc_sigma_file.values)
i_sort = np.argsort(retr_time)
retr_time = np.array(retr_time)[i_sort]
retr_cdnc = np.array(retr_cdnc)[i_sort]
retr_cdnc_sigma = np.array(retr_cdnc_sigma)[i_sort]
cap_cdnc = xr.Dataset(
    data_vars={
        'cdnc':('time',np.ma.masked_less_equal(retr_cdnc,0)),
        'cdnc_simga':('time',np.ma.masked_invalid(retr_cdnc_sigma))
    },
    coords={'time':retr_time},
    attrs={'units':'cm-3'}
)

# take 1 hr means
cap_n10_1hr = cap_n10.resample(time="1h").reduce(ma.mean)
cap_cdnc_1hr = cap_cdnc.resample(time="1h").reduce(ma.mean)

## Load model output
N10 has been calculated from UKCA aerosol fields in compute_ukca_n10.ipynb

In [ ]:
ocean_N10 = xr.open_dataarray('data/model/u-cu657_N10_CAP.nc')
coast_N10 = xr.open_dataarray('data/model/u-cr700_N10_CAP.nc')

# load CDNC
ocean_cdnc_grid = xr.open_dataarray('data/model/u-cu657_m01s00i075_CAP.nc')
coast_cdnc_grid = xr.open_dataarray('data/model/u-cr700_m01s00i075_CAP.nc')

# load liquid cloud fraction and water content
ocean_cfliq = xr.open_dataarray('data/model/u-cu657_m01s00i267_CAP.nc')
coast_cfliq = xr.open_dataarray('data/model/u-cr700_m01s00i267_CAP.nc')

# scale CDNC by cloud fraction to get in-cloud values
ocean_cdnc_cloud = ocean_cdnc_grid/ocean_cfliq
coast_cdnc_cloud = coast_cdnc_grid/coast_cfliq

# get column median CDNC value
ocean_cdnc = ocean_cdnc_cloud.median(dim='TH_1_60_eta_theta')*1e-6
coast_cdnc = coast_cdnc_cloud.median(dim='TH_1_60_eta_theta')*1e-6

# load liquid cloud water mass to compare some old analysis
ocean_liq_mass = xr.open_dataarray('data/model/u-cu657_m01s00i254_CAP.nc')
coast_liq_mass = xr.open_dataarray('data/model/u-cr700_m01s00i254_CAP.nc')
ocean_cdnc_grid_med = ocean_cdnc_grid.where(ocean_liq_mass>0).median(dim='TH_1_60_eta_theta', skipna=True)*1e-6
coast_cdnc_grid_med = coast_cdnc_grid.where(coast_liq_mass>0).median(dim='TH_1_60_eta_theta', skipna=True)*1e-6


### Load model output for ERA5_INITAL

In [ ]:
era5_code = 'u-cw894'
era5_N10 = xr.open_dataarray(f'data/model/{era5_code}_N10_CAP.nc')
# load CDNC
era5_cdnc_grid = xr.open_dataarray(f'data/model/{era5_code}_m01s00i075_CAP.nc')
# load liquid cloud fraction and water content
era5_cfliq = xr.open_dataarray(f'data/model/{era5_code}_m01s00i267_CAP.nc')
# scale CDNC by cloud fraction to get in-cloud values
era5_cdnc_cloud = era5_cdnc_grid/era5_cfliq
# get column median CDNC value
era5_cdnc = era5_cdnc_cloud.median(dim='TH_1_60_eta_theta')*1e-6
# load liquid cloud water mass to compare some old analysis
era5_liq_mass = xr.open_dataarray(f'data/model/{era5_code}_m01s00i254_CAP.nc')
era5_cdnc_grid_med = era5_cdnc_grid.where(era5_liq_mass>0).median(dim='TH_1_60_eta_theta', skipna=True)*1e-6


## Test sensitivity of CDCN to column median vs cloud base

In [ ]:
ocean_cloud_mask = (ocean_liq_mass>0)
ocean_cloud_mask_cf = (ocean_cfliq>0)
coast_cloud_mask = (coast_liq_mass>0)
coast_cloud_mask_cf = (coast_cfliq>0)

ocean_cloud_base = np.zeros(ocean_cloud_mask.shape)
ocean_cloud_base_cf = np.zeros(ocean_cloud_mask.shape)
coast_cloud_base = np.zeros(coast_cloud_mask.shape)
coast_cloud_base_cf = np.zeros(coast_cloud_mask.shape)
# set lowest level of cloud mask array to 1 if cloud is present
ocean_cloud_base[:,0] = (ocean_cloud_mask[:,0].values==1)
ocean_cloud_base_cf[:,0] = (ocean_cloud_mask_cf[:,0].values==1)
coast_cloud_base[:,0] = (coast_cloud_mask[:,0].values==1)
coast_cloud_base_cf[:,0] = (coast_cloud_mask_cf[:,0].values==1)
# for levels above surface, set to 1 if there is cloud present and no cloud present in level above
ocean_cloud_base[:,1:] = (ocean_cloud_mask[:,1:].values==1) * (ocean_cloud_mask[:,:-1].values==0)
ocean_cloud_base_cf[:,1:] = (ocean_cloud_mask_cf[:,1:].values==1) * (ocean_cloud_mask_cf[:,:-1].values==0)
coast_cloud_base[:,1:] = (coast_cloud_mask[:,1:].values==1) * (coast_cloud_mask[:,:-1].values==0)
coast_cloud_base_cf[:,1:] = (coast_cloud_mask_cf[:,1:].values==1) * (coast_cloud_mask_cf[:,:-1].values==0)

# make array with CDNC values at cloud base
ocean_cdnc_cloud_base = ma.zeros((ocean_cloud_mask.shape[0]))
ocean_cdnc_cloud_base_cf = ma.zeros((ocean_cloud_mask.shape[0]))
for i in np.arange(ocean_cdnc_cloud_base.shape[0]):
    zcloud_base = np.nonzero(ocean_cloud_base[i])[0]
    if zcloud_base.size>0:
        ocean_cdnc_cloud_base[i] = ocean_cdnc_cloud.values[i,zcloud_base[0]]
    zcloud_base = np.nonzero(ocean_cloud_base_cf[i])[0]
    if zcloud_base.size>0:
        ocean_cdnc_cloud_base_cf[i] = ocean_cdnc_cloud.values[i,zcloud_base[0]]

coast_cdnc_cloud_base = ma.zeros((coast_cloud_mask.shape[0]))
coast_cdnc_cloud_base_cf = ma.zeros((coast_cloud_mask.shape[0]))
for i in np.arange(coast_cdnc_cloud_base.shape[0]):
    zcloud_base = np.nonzero(coast_cloud_base[i])[0]
    if zcloud_base.size>0:
        coast_cdnc_cloud_base[i] = coast_cdnc_cloud.values[i,zcloud_base[0]]
    zcloud_base = np.nonzero(coast_cloud_base_cf[i])[0]
    if zcloud_base.size>0:
        coast_cdnc_cloud_base_cf[i] = coast_cdnc_cloud.values[i,zcloud_base[0]]

### Test difference between using LWC>0 as cloud mask or CF>0 as cloud mask

In [ ]:
fig = plt.figure(figsize=(15,10), dpi=300)
ax1, ax2 = fig.subplots(2,1)
ax1.plot(ocean_cdnc_cloud.T1HR, ocean_cdnc_cloud_base, label='LWC')
ax1.plot(ocean_cdnc_cloud.T1HR, ocean_cdnc_cloud_base_cf, label='CF')
ax1.legend()
ax2.plot(coast_cdnc_cloud.T1HR, coast_cdnc_cloud_base, label='LWC')
ax2.plot(coast_cdnc_cloud.T1HR, coast_cdnc_cloud_base_cf, label='CF')


### Test difference between cloud base CDNC and column median

In [ ]:
fig = plt.figure(figsize=(15,10), dpi=300)
ax1, ax2 = fig.subplots(2,1)
ocean_cdnc_subset = cap_cdnc_1hr.sel(time=ocean_cdnc.T1HR.values, method='nearest')
coast_cdnc_subset = cap_cdnc_1hr.sel(time=coast_cdnc.T1HR.values, method='nearest')
ax1.plot(ocean_cdnc_subset.time, ocean_cdnc_subset.cdnc, 'k', label='CAPRICORN')
l, = ax1.plot(ocean_cdnc.T1HR, ocean_cdnc, label='Median', ls='dashed')
ax1.plot(ocean_cdnc_cloud.T1HR, 1e-6*ocean_cdnc_cloud_base, c=l.get_color(), label='Cloud base')
ax1.set_yscale('log')
ax1.set_title('Ocean', loc='left')
ax1.set_ylabel('CDNC [cm-3]')
ax1.legend()
ax2.plot(coast_cdnc_subset.time, coast_cdnc_subset.cdnc, 'k')
l, = ax2.plot(coast_cdnc.T1HR, coast_cdnc, label='Median', c=l.get_color(), ls='dashed')
ax2.plot(coast_cdnc_cloud.T1HR, 1e-6*coast_cdnc_cloud_base, label='LWC')
ax2.set_yscale('log')
ax2.set_title('Coast', loc='left')
ax2.set_ylabel('CDNC [cm-3]')
filename = 'figures/cdnc_cloud_base_sensitivity.png'
plt.savefig(filename, bbox_inches='tight', facecolor='white')

## Plot figure 2

In [ ]:
fig = plt.figure(figsize=(20*fprops.in_cm,10*fprops.in_cm), dpi=fprops.dpi)
axes = fig.subplots(2,2, gridspec_kw={'width_ratios':[5,4],
                                     'wspace':0.025,
                                     'hspace':0.4})
ocean_cpc_subset = cap_n10_1hr.sel(time=ocean_N10.T1HR.values, method='nearest').n10
coast_cpc_subset = cap_n10_1hr.sel(time=coast_N10.T1HR.values, method='nearest').n10
ocean_retr_subset = cap_cdnc_1hr.sel(time=ocean_cdnc.T1HR.values, method='nearest').cdnc
coast_retr_subset = cap_cdnc_1hr.sel(time=coast_cdnc.T1HR.values, method='nearest').cdnc
# ax1: cu657 (control) N10
ax1 = axes[0,0]
ax1.plot(ocean_cpc_subset.time.values, ocean_cpc_subset.values, 'k', label='CAPRICORN-2')
ax1.plot(ocean_N10.T1HR.values, ocean_N10.values[:,0]*1e-6, label='CONTROL', color='tab:blue')
ax1.set_title('(a)', loc='left', fontsize=fprops.label_fs)
ax1.set_title('Ocean', fontsize=fprops.label_fs)
ax1.set_ylabel('N$_{{10}}$ [cm$^{{-3}}$]', fontsize=fprops.ax_label_fs, x=0.07)
# ax2: cu657 (control) CDNC
ax2 = axes[1,0]
ax2.axhline(y=150, c='grey')
ax2.plot(ocean_retr_subset.time.values, ocean_retr_subset.values, 'k')
ax2.plot(ocean_cdnc.T1HR.values, ocean_cdnc.values, color='tab:blue')
ax2.set_title('(c)', loc='left', fontsize=fprops.label_fs)
ax2.set_ylabel('CDNC [cm$^{{-3}}$]', fontsize=fprops.ax_label_fs, x=0.07)
# ax3: cr700 (control) N10
ax3 = axes[0,1]
ax3.plot(coast_cpc_subset.time.values, coast_cpc_subset.values, 'k')
ax3.plot(coast_N10.T1HR.values, coast_N10.values[:,0]*1e-6, color='tab:blue')
ax3.set_title('(b)', loc='left', fontsize=fprops.label_fs)
ax3.set_title('Coastal', fontsize=fprops.label_fs)
# ax4: cr700 (control) CDNC
ax4 = axes[1,1]
ax4.axhline(y=150, c='grey')
ax4.plot(coast_retr_subset.time.values, coast_retr_subset.values, 'k')
ax4.plot(coast_cdnc.T1HR.values, coast_cdnc.values, color='tab:blue')
ax4.set_title('(d)', loc='left', fontsize=fprops.label_fs)
# adjust labels
xtick_fmt = '%d-%b'
for ax in [ax1,ax3]:
    ax.set_ylim(1e0,1.2e3)
for ax in [ax2,ax4]:
    ax.set_ylim(0.3e-1,0.2e4)
for ax in [ax1,ax2,ax3,ax4]:
    ax.set_yscale('log')
    ax.tick_params(axis='x', labelsize=fprops.ax_fs)
    ax.tick_params(axis='y', labelsize=fprops.ax_fs)
    ax.xaxis.set_major_formatter(mdates.DateFormatter(xtick_fmt))
for ax in [ax3,ax4]:
    ax.set_yticklabels([])
    ax.set_yticklabels([])
    xlim = ax.get_xlim()
ax1.legend(fontsize=fprops.legend_fs)
filename = 'figures/figure2.png'
plt.savefig(filename, bbox_inches='tight',
            facecolor='white')

In [ ]:
print('N10 MEANS')
print(f'u-cu657: {ma.mean(ocean_N10.values[:,0]*1e-6):.2f}, {ma.median(ocean_N10.values[:,0]*1e-6):.2f}')
print(f'obs: {ma.mean(ma.masked_invalid(ocean_cpc_subset.values)):.2f}, {ma.median(ma.masked_invalid(ocean_cpc_subset.values)):.2f}')
print(f'u-cr700: {ma.mean(coast_N10.values[:,0]*1e-6):.2f}, {ma.median(coast_N10.values[:,0]*1e-6):.2f}')
print(f'obs: {ma.mean(ma.masked_invalid(coast_cpc_subset.values)):.2f}, {ma.median(ma.masked_invalid(coast_cpc_subset.values)):.2f}')

print('\nCDNC MEANS')
print(f'u-cu657: {ma.mean(ocean_cdnc.values):.2f}, {ma.median(ocean_cdnc.values):.2f}')
print(f'obs: {ma.mean(ma.masked_invalid(ocean_retr_subset.values)):.2f}, {ma.median(ma.masked_invalid(ocean_retr_subset.values)):.2f}')
print(f'u-cr700: {ma.mean(ma.masked_invalid(coast_cdnc.values)):.2f}, {ma.median(ma.masked_invalid(coast_cdnc.values)):.2f}')
print(f'obs: {ma.mean(ma.masked_invalid(coast_retr_subset.values)):.2f}, {ma.median(ma.masked_invalid(coast_retr_subset.values)):.2f}')

print('\nCDNC MIN/MAX')
print(f'u-cu657: {ma.amin(ocean_cdnc.values):.2f}-{ma.amax(ocean_cdnc.values):.2f}')
print(f'obs: {ma.amin(ma.masked_invalid(ocean_retr_subset.values)):.2f}-{ma.amax(ma.masked_invalid(ocean_retr_subset.values)):.2f}')
print(f'u-cr700: {ma.amin(ma.masked_invalid(coast_cdnc.values)):.2f}-{ma.amax(ma.masked_invalid(coast_cdnc.values)):.2f}')
print(f'obs: {ma.amin(ma.masked_invalid(coast_retr_subset.values)):.2f}-{ma.amax(ma.masked_invalid(coast_retr_subset.values)):.2f}')


## Plot N10 distributions

In [ ]:
fig = plt.figure(figsize=(10*fprops.in_cm,5*fprops.in_cm), dpi=fprops.dpi)
axes = fig.subplots(1, 2)
# ax1: ocean
ax = axes[0]
# mask CPC anomalies
ocean_cpc_masked = ma.masked_invalid(ocean_cpc_subset).compressed()
ax.boxplot([ocean_N10.values[:,0]*1e-6, ocean_cpc_masked],
           positions=[1, 2], patch_artist=True, showfliers=False,
           boxprops=dict(color='tab:blue'), medianprops=dict(color='black'))
ax.set_title('Ocean', fontsize=fprops.label_fs)
ax.tick_params(axis='both', labelsize=fprops.ax_fs)
yticks = ax.get_yticks()
ax.set_ylabel('N10 [cm$^{{-3}}$]', fontsize=fprops.ax_label_fs)
ax.set_xticks([1, 2])
ax.set_xticklabels(['Model', 'CPC'])
ylim = ax.get_ylim()

# ax2: coast
ax = axes[1]
# mask CPC anomalies
coast_cpc_masked = ma.masked_invalid(coast_cpc_subset).compressed()
ax.boxplot([coast_N10.values[:,0]*1e-6, coast_cpc_masked],
           positions=[1, 2], patch_artist=True, showfliers=False,
           boxprops=dict(color='tab:blue'), medianprops=dict(color='black'))
ax.tick_params(axis='both', labelsize=fprops.ax_fs)
ax.set_xticklabels(['Model', 'CPC'])
ax.set_ylim(ylim)
ax.set_yticklabels([])
ax.set_title('Coastal', fontsize=fprops.label_fs)
plt.savefig('figures/figureS1.png', bbox_inches='tight',
            facecolor='white')


## Plot figures for response to reviews

### Plot CONTROL and ERA5_INITIAL N10

In [ ]:
fig = plt.figure(figsize=(20*fprops.in_cm,10*fprops.in_cm), dpi=fprops.dpi)
axes = fig.subplots(2,1)
coast_cpc_subset = cap_n10_1hr.sel(time=coast_N10.T1HR.values, method='nearest')
coast_retr_subset = cap_cdnc_1hr.sel(time=coast_cdnc.T1HR.values, method='nearest')
# ax1: N10
ax1 = axes[0]
ax1.plot(coast_cpc_subset.time.values, coast_cpc_subset.n10.values, 'k', label='CAPRICORN-2')
ax1.plot(coast_N10.T1HR.values, coast_N10.values[:,0]*1e-6, color='tab:blue', label='CONTROL')
ax1.plot(era5_N10.T1HR.values, era5_N10.values[:,0]*1e-6, color='tab:green', label='ERA5_INITIAL')
ax1.set_ylabel('N$_{{10}}$ [cm$^{{-3}}$]', fontsize=fprops.ax_label_fs, x=0.07)
# ax2: CDNC
ax2 = axes[1]
ax2.plot(coast_retr_subset.time.values, coast_retr_subset.cdnc.values, 'k')
ax2.plot(coast_cdnc.T1HR.values, coast_cdnc.values, color='tab:blue')
ax2.plot(era5_cdnc.T1HR.values, era5_cdnc.values, color='tab:green')
ax2.set_ylabel('CDNC [cm$^{{-3}}$]', fontsize=fprops.ax_label_fs, x=0.07)
# adjust labels
xtick_fmt = '%d-%b'
ax1.set_ylim(1e0,1.2e3)
ax2.set_ylim(0.3e-1,0.2e4)
for ax in [ax1,ax2]:
    ax.set_yscale('log')
    ax.tick_params(axis='x', labelsize=fprops.ax_fs)
    ax.tick_params(axis='y', labelsize=fprops.ax_fs)
    ax.xaxis.set_major_formatter(mdates.DateFormatter(xtick_fmt))
ax1.legend(fontsize=fprops.legend_fs)
filename = 'figures/N10_CONTROL_ERA5_INITIAL.png'
plt.savefig(filename, bbox_inches='tight',
            facecolor='white')

### Plot vertical variability on time series of CDNC

In [ ]:
ocean_cpc_subset = cap_n10_1hr.sel(time=ocean_N10.T1HR.values, method='nearest').n10
coast_cpc_subset = cap_n10_1hr.sel(time=coast_N10.T1HR.values, method='nearest').n10
ocean_retr_subset = cap_cdnc_1hr.sel(time=ocean_cdnc.T1HR.values, method='nearest').cdnc
coast_retr_subset = cap_cdnc_1hr.sel(time=coast_cdnc.T1HR.values, method='nearest').cdnc
zlevel = 1000
theta_level = zlevel/const.ztop
ocean_N10_1km = ocean_N10.sel(TH_1_60_eta_theta=slice(0,theta_level))
ocean_N10_1km_mean = ocean_N10_1km.mean(dim='TH_1_60_eta_theta')
ocean_N10_1km_std = ocean_N10_1km.std(dim='TH_1_60_eta_theta')
ocean_N10_1km_lower = ocean_N10_1km_mean - 0.5*ocean_N10_1km_std
ocean_N10_1km_upper = ocean_N10_1km_mean + 0.5*ocean_N10_1km_std
coast_N10_1km = coast_N10.sel(TH_1_60_eta_theta=slice(0,theta_level))
coast_N10_1km_mean = coast_N10_1km.mean(dim='TH_1_60_eta_theta')
coast_N10_1km_std = coast_N10_1km.std(dim='TH_1_60_eta_theta')
coast_N10_1km_lower = coast_N10_1km_mean - 0.5*coast_N10_1km_std
coast_N10_1km_upper = coast_N10_1km_mean + 0.5*coast_N10_1km_std

fig = plt.figure(figsize=(20*fprops.in_cm,5*fprops.in_cm), dpi=fprops.dpi)
axes = fig.subplots(1,2, gridspec_kw={'width_ratios':[5,4], 'wspace':0.025})
# ax1: cu657 (control) N10
ax1 = axes[0]
ax1.plot(ocean_cpc_subset.time.values, ocean_cpc_subset.values, 'k', label='CAPRICORN-2')
ax1.plot(ocean_N10.T1HR.values, ocean_N10.values[:,0]*1e-6, ls='dashed', label='CONTROL surface', color='tab:blue')
ax1.plot(ocean_N10.T1HR.values, ocean_N10_1km_mean*1e-6, label=f'CONTROL 0-{zlevel} m mean', color='tab:blue')
ax1.fill_between(ocean_N10.T1HR.values, ocean_N10_1km_lower*1e-6, ocean_N10_1km_upper*1e-6, color='tab:blue', alpha=0.5)
ax1.set_title('(a)', loc='left', fontsize=fprops.label_fs)
ax1.set_title('Ocean', fontsize=fprops.label_fs)
ax1.set_ylabel('N$_{{10}}$ [cm$^{{-3}}$]', fontsize=fprops.ax_label_fs, x=0.07)
# ax3: cr700 (control) N10
ax3 = axes[1]
ax3.plot(coast_cpc_subset.time.values, coast_cpc_subset.values, 'k', label='CAPRICORN')
ax3.plot(coast_N10.T1HR.values, coast_N10.values[:,0]*1e-6, ls='dashed', color='tab:blue', label='CONTROL_surface')
ax3.plot(coast_N10.T1HR.values, coast_N10_1km_mean*1e-6, label=f'CONTROL 0-{zlevel} m mean', color='tab:blue')
ax3.fill_between(coast_N10.T1HR.values, coast_N10_1km_lower*1e-6, coast_N10_1km_upper*1e-6, color='tab:blue', alpha=0.5)
ax3.set_title('(b)', loc='left', fontsize=fprops.label_fs)
ax3.set_title('Coastal', fontsize=fprops.label_fs)
# adjust labels
xtick_fmt = '%d-%b'
for ax in [ax1,ax3]:
    ax.set_ylim(1e0,1.2e3)
    ax.set_yscale('log')
    ax.tick_params(axis='x', labelsize=fprops.ax_fs)
    ax.tick_params(axis='y', labelsize=fprops.ax_fs)
    ax.xaxis.set_major_formatter(mdates.DateFormatter(xtick_fmt))
ax3.set_yticklabels([])
ax3.legend(fontsize=fprops.legend_fs)
filename = 'figures/N10_CDNC_vertical_variability.png'
plt.savefig(filename, bbox_inches='tight',
            facecolor='white')

### Plot vertical profiles of model N10

In [ ]:
levels = [1e0,5e0,1e1,5e1,1e2,5e2]
cmap = matplotlib.colormaps['viridis']
norm = matplotlib.colors.BoundaryNorm(levels, cmap.N, extend='both')
# ocean
fig = plt.figure(figsize=(15,10), dpi=300)
ax1, ax2 = fig.subplots(2,1)
C = ax1.pcolormesh(ocean_N10.T1HR, (ocean_N10.TH_1_60_eta_theta*const.ztop)/1000,
                   ocean_N10.T*1e-6, shading='nearest',
                   norm=norm)
ax1.contour(ocean_cfliq.T1HR, (ocean_cfliq.TH_1_60_eta_theta*const.ztop)/1000,
            ocean_cfliq.values.T*100, levels=[5], colors='white')
ax1.axhline(y=zlevel/1000, c='white', ls='dashed', lw=0.5)
ax1.set_title('Ocean', loc='left')
# coast
C = ax2.pcolormesh(coast_N10.T1HR, (coast_N10.TH_1_60_eta_theta*const.ztop)/1000,
                   coast_N10.T*1e-6, shading='nearest',
                   norm=norm)
ax2.contour(coast_cfliq.T1HR, (coast_cfliq.TH_1_60_eta_theta*const.ztop)/1000,
            coast_cfliq.values.T*100, levels=[5], colors='white')
plt.colorbar(C, label=r'N10 [cm$^{{-3}}$]', ax=[ax1,ax2], shrink=0.6)
ax2.axhline(y=zlevel/1000, c='white', ls='dashed', lw=0.5)
ax2.set_title('Coast', loc='left')
fig.supylabel('Altitude [km]', x=0.075)
filename = 'figures/n10_vertical_profiles.png'
plt.savefig(filename, bbox_inches='tight',
            facecolor='white')